In [ ]:
import pickle
import warnings
import numpy as np
from scipy import spatial
from scipy.io import loadmat
from scipy.stats import pearsonr
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")


In [ ]:
# pickle load the file ensemble.pkl
with open("ensemble_wt_avg.pkl", "rb") as f:
    ensemble = pickle.load(f)


In [ ]:
def pairwise_accuracy(actual, predicted):
    true = 0
    total = 0
    for i in range(0, len(actual)):
        for j in range(i + 1, len(actual)):
            total += 1

            s1 = actual[i]
            s2 = actual[j]
            b1 = predicted[i]
            b2 = predicted[j]

            result1 = spatial.distance.cosine(s1, b1)
            result2 = spatial.distance.cosine(s2, b2)
            result3 = spatial.distance.cosine(s1, b2)
            result4 = spatial.distance.cosine(s2, b1)

            if result1 + result2 < result3 + result4:
                true += 1

    return true / total


In [ ]:
def pearcorr(actual, predicted):
    corr = []
    for i in range(0, len(actual)):
        corr.append(np.corrcoef(actual[i], predicted[i])[0][1])
    return np.mean(corr)


In [ ]:
ROIS = [
    "language_lh",
    "language_rh",
    "vision_body",
    "vision_face",
    "vision_object",
    "vision_scene",
    "vision",
    "dmn",
    "task",
]
subjects = ["P01", "M02", "M04", "M07", "M15"]
layers_bert = [
    "block1",
    "block2",
    "block3",
    "block4",
    "block5",
    "block6",
    "block7",
    "block8",
    "block9",
    "block10",
    "block11",
    "block12",
    "fc",
]
layers_bart = ["fc"]


In [ ]:
def evaluate(Y, roi, subject):
    dataset_Y = np.array(Y.copy())
    _, y_test, _, y_pred = train_test_split(
        dataset_Y, ensemble[subject][roi], test_size=0.2, random_state=42
    )
    acc = pairwise_accuracy(y_test, y_pred)
    corr = pearcorr(y_test, y_pred)

    return acc, corr


In [ ]:
output = {}
output["2v2"] = {}
output["pear"] = {}

for subject in subjects:
    print(subject)
    output["2v2"][subject] = {}
    output["pear"][subject] = {}
    fmri = get_subject_data(subject)

    for roi in ROIS:
        print(roi)
        voxels = np.array(fmri[roi])
        acc, corr = evaluate(voxels, roi, subject)
        output["2v2"][subject][roi] = acc
        output["pear"][subject][roi] = corr


In [ ]:
ensemble_output = {}
ensemble_output["2v2"] = {}
ensemble_output["pear"] = {}
for roi in ROIS:
    ensemble_output["2v2"][roi] = {}
    ensemble_output["pear"][roi] = {}
    for subject in subjects:
        ensemble_output["2v2"][roi][subject] = output["2v2"][subject][roi]
        ensemble_output["pear"][roi][subject] = output["pear"][subject][roi]


In [ ]:
with open(f"results/results_ensemble_wt_avg.pkl", "wb") as f:
    pickle.dump(ensemble_output, f)
